In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import csv 
from sklearn.model_selection import train_test_split
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/USYD/2021 S1/INFO4001/zhengmix4uneq-RawCount-modified.csv',header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,10,11,18,22,23,26,27,30,32,34,35,43,51,52,53,56,57,61,62,63,69,74,78,84,85,87,88,89,90,91,93,94,96,97,99,100,102,104,105,107,111,113,114,117,121,123,124,125,127,129,130,132,133,137,138,143,146,147,150,151,158,159,161,164,167,168,169,171,172,175,176,179,183,186,188,193,194,198,200,202,204,209,211,212,214,216,217,219,222,224,225,226,229,230,234,241,242,244,245,250,256,257,258,259,267,269,270,271,273,274,277,278,279,280,282,284,285,286,287,288,289,290,293,294,295,296,298,299,300,301,303,304,305,306,307,308,310,311,312,313,315,316,317,318,319,321,323,324,325,326,327,328,329,330,331,332,333,336,337,338,339,340,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,367,368,369,370,372,373,374,375,376,377,378,379,380,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,398,400,401,402,403,404,405,406,408,409,410,412,413,414,415,416,418,419,42

In [ ]:
def shuffle_data(x):
    
    # save header
    tmp_head = x[0]

    #remove header 
    x = x[1:]

    # shuffle by row 
    np.random.shuffle(x)

    #stack header
    x = np.vstack((tmp_head,x))

    # shuffle by column after T transformation 
    x = x.T 

    tmp_head = x[0]
    x = x[1:]

    np.random.shuffle(x)

    x = np.vstack((tmp_head,x))

    return x 

In [ ]:
def remove_invalid_val(mt):
    inval_lst= ['nan','alpha.contaminated', 'beta.contaminated', 
    'delta.contaminated', 'Excluded', 'gamma.contaminated', 'miss', 'NA',
    'not applicable', 'unclassified', 'unknown', 'Unknown', 'zothers']
    print("before shape {}".format(mt.shape))
    lst = mt[0].astype(str)
    d_lst=[]

    for i in range(len(lst)):
        for j in range(len(inval_lst)):

            if lst[i] == inval_lst[j]:
                d_lst.append(i)
    print(len(d_lst))
    mt = np.delete(mt,d_lst,axis=1)
    print("after shape {}".format(mt.shape))
    return mt

In [ ]:
import re 

def trim_cell_type(mt):
    pattern = r'[0-9]'
    for i in range(mt.shape[1]):
        mt[0][i] = re.sub(pattern, '',  mt[0][i])
    
    return mt

In [ ]:
# data preprocessing and data split 

mt = df.to_numpy()
#mt = mt.T
mt = remove_invalid_val(mt)


#trim cell type numbers
mt =  trim_cell_type(mt)


#shuffle dataset by rows and cols
mt = shuffle_data(mt)

# get type for each sample 
mt_Y = mt.T[0][1:]

mt1 = mt[:,1:]
mt_X = mt1[1:]
mt_X = mt_X.astype('float64')


unique_classes = np.unique(mt_Y)

# split data to train and test 
x_train, x_test, y_train, y_test = train_test_split(
    mt_X, mt_Y, test_size=0.2,stratify=mt_Y, random_state=42)

# x_train, x_test, y_train, y_test = train_test_split(
#     mt_X, mt_Y, test_size=0.2, random_state=42)


print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(unique_classes)
print(len(unique_classes))

before shape (20017, 1962)
634
after shape (20017, 1328)
(1061, 20016) (266, 20016) (1061,) (266,)
['endothelial cell' 'leukocyte' 'pancreatic A cell' 'pancreatic D cell'
 'pancreatic PP cell' 'pancreatic acinar cell' 'pancreatic ductal cell'
 'pancreatic stellate cell' 'type B pancreatic cell']
9


In [ ]:
#Different normalisation method: raw, CPM, minmax(CPM)

#cpm only
tmp_train = np.nan_to_num(x_train)
tmp_test = np.nan_to_num(x_test)
sum_train = np.sum(tmp_train,axis=1)
sum_train = sum_train.reshape(sum_train.shape[0],-1)
sum_test = np.sum(tmp_test,axis=1)
sum_test = sum_test.reshape(sum_test.shape[0],-1)
#print(tmp_train.shape)
#print(np.sum(tmp_train,axis=1))

#cpm_x_train = np.divide(tmp_train, np.sum(tmp_train,axis=1))



#print(a2.shape)
#cpm_x_train = tmp_train /a2
cpm_x_train = np.divide(tmp_train, sum_train)
cpm_x_test =  np.divide(tmp_test, sum_test)
print(cpm_x_train.shape)



#cpm with minmax
mnx = MinMaxScaler()

mnx_x_train = mnx.fit_transform(cpm_x_train)

mnx_x_test = mnx.transform(cpm_x_test)



(1061, 20016)


In [ ]:

label = unique_classes.tolist()


n_lst=[]
for i in range(len(y_train)):
  for j in range(len(label)):
    if y_train[i] == label[j]:

      n_lst.append(j)

y_new_train = np.array(n_lst,dtype='float64')

n_lst=[]
for i in range(len(y_test)):
  for j in range(len(label)):
    if y_test[i] == label[j]:
      n_lst.append(j)

y_new_test = np.array(n_lst,dtype='float64')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train,y_new_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
y_pred = neigh.predict(x_test)



In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

out_acc = accuracy_score(y_new_test, y_pred)
print("Raw data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("Raw data F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("Raw data F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("Raw data F1 Score(weighted): {}".format(round(out_f1,4)))

Raw data Accuracy: 0.8684
Raw data F1 Score(micro): 0.8684
Raw data F1 Score(macro): 0.7647
Raw data F1 Score(weighted): 0.8643


In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(cpm_x_train,y_new_train)
y_pred = neigh.predict(cpm_x_test)

out_acc = accuracy_score(y_new_test, y_pred)
print("CPM data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("CPM data  F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("CPM data  F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("CPM data  F1 Score(weighted): {}".format(round(out_f1,4)))


CPM data Accuracy: 0.8985
CPM data  F1 Score(micro): 0.8985
CPM data  F1 Score(macro): 0.8163
CPM data  F1 Score(weighted): 0.8985


In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(mnx_x_train,y_new_train)
y_pred = neigh.predict(mnx_x_test)

out_acc = accuracy_score(y_new_test, y_pred)
print("mixmax(CPM) data Accuracy: {}".format(round(out_acc,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='micro')
print("mixmax(CPM) data  F1 Score(micro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='macro')
print("mixmax(CPM) data  F1 Score(macro): {}".format(round(out_f1,4)))

out_f1 = f1_score(y_new_test, y_pred,average ='weighted')
print("mixmax(CPM) data  F1 Score(weighted): {}".format(round(out_f1,4)))

mixmax(CPM) data Accuracy: 0.4586
mixmax(CPM) data  F1 Score(micro): 0.4586
mixmax(CPM) data  F1 Score(macro): 0.2538
mixmax(CPM) data  F1 Score(weighted): 0.3508
